<a href="https://colab.research.google.com/github/CristinaMarsh/Data/blob/main/Reanalysis/Basic_data_structures_of_xarray_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://towardsdatascience.com/basic-data-structures-of-xarray-80bab8094efa

In [ ]:
# customary imports
import numpy as np
import pandas as pd
import xarray as xr

In [ ]:
np.random.seed(123)
size = 4
temperature = 15 + 10 * np.random.randn(size)
lat = np.random.uniform(low=-90,high=90,size=size)
lon = np.random.uniform(low=-180,high=180,size=size)

temperature, lat, lon = np.around([temperature,lat,lon],decimals=2)

In [ ]:
df = pd.DataFrame({'temperature':temperature,'lat':lat,'lon':lon})
df.head()

,temperature,lat,lon
0,4.14,39.50,-6.86
1,24.97,-13.84,-38.84
2,17.83,86.54,-56.46
3,-0.06,33.27,82.46


We will create a `DataArray` from this data, let’s have a look at four ways to do that:

from a pandas Series<br>
from a pandas DataFrame<br>
using the DataArray constructor<br>
using the DataArray constructor with projected coordinates<br>


### Creating DataArray from Series
We’ll create a pandas Series and then a DataArray. Since we want to represent 2 dimensions in our data, we will create a series with a 2 level multi-index:

In [ ]:
idx = pd.MultiIndex.from_arrays(arrays=[lat,lon],names=['lat','lon'])
s = pd.Series(data=temperature,index=idx)
s

lat     lon   
 39.50  -6.86      4.14
-13.84  -38.84    24.97
 86.54  -56.46    17.83
 33.27   82.46    -0.06
dtype: float64

In [ ]:
da = xr.DataArray.from_series(s)
da

<xarray.DataArray (lat: 4, lon: 4)>
array([[  nan, 24.97,   nan,   nan],
       [  nan,   nan,   nan, -0.06],
       [  nan,   nan,  4.14,   nan],
       [17.83,   nan,   nan,   nan]])
Coordinates:
  * lat      (lat) float64 -13.84 33.27 39.5 86.54
  * lon      (lon) float64 -56.46 -38.84 -6.86 82.46

This is what the data array looks like when printed.

### Creating a DataArray from DataFrame
We can provide the DataArray constructor with a pandas DataFrame. It will consider the index of the data frame as the first dimension and the columns as the second. If the index or the columns have multiple levels xarray will create nested dimensions.

Since we want latitude and longitude as our dimensions, the smartest way to achieve this would be to pivot our data frame using latitude as index and longitude as columns:

In [ ]:
df_pv = df.pivot(index='lat',columns='lon')

#drop first level of columns as it's not necessary

df_pv = df_pv.droplevel(0,axis=1)
df_pv

lon,-56.46,-38.84,-6.86,82.46
lat,,,,
-13.84,NaN,24.97,NaN,NaN
33.27,NaN,NaN,NaN,-0.06
39.50,NaN,NaN,4.14,NaN
86.54,17.83,NaN,NaN,NaN


In [ ]:
da = xr.DataArray(data=df_pv)
da

<xarray.DataArray (lat: 4, lon: 4)>
array([[  nan, 24.97,   nan,   nan],
       [  nan,   nan,   nan, -0.06],
       [  nan,   nan,  4.14,   nan],
       [17.83,   nan,   nan,   nan]])
Coordinates:
  * lat      (lat) float64 -13.84 33.27 39.5 86.54
  * lon      (lon) float64 -56.46 -38.84 -6.86 82.46

### Creating a DataArray using the constructor
So we’ve seen two ways to create a DataArray from pandas objects. Now let’s see how we can create a DataArray manually. Since we want to represent 2 dimensions in our data, the data should be shaped in a 2-dimensional array so we can pass it directly to the DataArray constructor.

We’ll use the data from the pivoted data frame, then we’ll need to specify the coordinates and dimensions explicitly:

In [ ]:
# get pivoted data as 2-dimensional array (4,4)

temperature_2d = df_pv.values
da = xr.DataArray(data=temperature_2d,dims=['lat','lon'],coords=[lat,lon])

da

<xarray.DataArray (lat: 4, lon: 4)>
array([[  nan, 24.97,   nan,   nan],
       [  nan,   nan,   nan, -0.06],
       [  nan,   nan,  4.14,   nan],
       [17.83,   nan,   nan,   nan]])
Coordinates:
  * lat      (lat) float64 39.5 -13.84 86.54 33.27
  * lon      (lon) float64 -6.86 -38.84 -56.46 82.46

### Creating a DataArray using the constructor with projected coordinates
We’ll check out one final way to create a DataArray, with projected coordinates. They might be useful in some cases, but they have one disadvantage, which is the coordinates have no clear interpretability. The big advantage of using them is that we can pass to the DataArray constructor arrays of the same shape, for both data and coordinates, without having to think about pivoting our data before.

In our case, we have temperature data, and we have two dimensions: latitude and longitude, so we can represent our data in a 2-dimensional array of any shape (it doesn’t have to be pivoted) and then provide the constructor with 2 coordinate arrays of the same shape for latitude and longitude:

In [ ]:
np.random.seed(123)
shape = (1,4)    # needs to be 2-dimensional, could be (2,2), (4,1)

# all three arrays have the same shape
temperature = 15 + 10 * np.random.randn(*shape)
lat = np.random.uniform(low=-90, high=90, size=shape)
lon = np.random.uniform(low=-180, high=180, size=shape)

# round to two digits after decimal point
temperature, lat , lon = np.around([temperature, lat, lon], decimals=2)

In [ ]:
da = xr.DataArray(data=temperature,
                  coords={"lat": (["x","y"], lat),
                          "lon": (["x","y"], lon)},
                  dims=["x","y"])
da

<xarray.DataArray (x: 1, y: 4)>
array([[ 4.14, 24.97, 17.83, -0.06]])
Coordinates:
    lat      (x, y) float64 39.5 -13.84 86.54 33.27
    lon      (x, y) float64 -6.86 -38.84 -56.46 82.46
Dimensions without coordinates: x, y

### 3 dimensions
Now let’s create another dimension! Let’s create temperature data for 2 days, not 1 but 2!

Like before for every day we need a 2-dimensional (latitude and longitude) array for temperature values. To represent data for 2 days we will want to stack the daily arrays together, resulting in a 3-dimensional array:

In [ ]:
np.random.seed(123)

temperature_3d = 15 + 10 * np.random.randn(1,4,2)    # 3-dimensional
lat = np.random.uniform(low=-90, high=90, size=(1,4))
lon = np.random.uniform(low=-180, high=180, size=(1,4))

# round to two digits after decimal point
temperature_3d = np.around(temperature_3d, decimals=2)
lat , lon = np.around([lat, lon], decimals=2)

In [ ]:
da = xr.DataArray(data=temperature_3d,
                  coords={"lat": (["x","y"], lat),
                          "lon": (["x","y"], lon), 
                          "day": ["day1","day2"]},
                  dims=["x","y","day"])
da

<xarray.DataArray (x: 1, y: 4, day: 2)>
array([[[ 4.14, 24.97],
        [17.83, -0.06],
        [ 9.21, 31.51],
        [-9.27, 10.71]]])
Coordinates:
    lat      (x, y) float64 -28.23 41.23 -11.06 -79.26
    lon      (x, y) float64 -36.7 85.68 -114.3 -116.8
  * day      (day) <U4 'day1' 'day2'
Dimensions without coordinates: x, y

We can also create the same thing using a pandas Series with a 3-level multi-index. To create a Series we will need to flatten the data, which means to make it 1-dimensional:

In [ ]:
# make data 1-dimensional
temperature_1d = temperature_3d.flatten("F")
lat = lat.flatten()
lon = lon.flatten()
day = ["day1","day2"]

In [ ]:
idx = pd.MultiIndex(levels=[day,lat,lon], 
                    codes=[[0]*4 + [1]*4, list(range(4))*2, list(range(4))*2], 
                    names=["day","lat","lon"])

s = pd.Series(temperature_1d, index=idx)

### Dataset
Up until this point, we only dealt with temperature data. Let’s add pressure data:

In [ ]:
np.random.seed(123)

# 3-dimensional temperature and pressure data
temperature_3d = 15 + 10 * np.random.randn(1,4,2)
pressure_3d = 1013 + 10 * np.random.randn(1,4,2)
lat = np.random.uniform(low=-90, high=90, size=(1,4))
lon = np.random.uniform(low=-180, high=180, size=(1,4))

# round to two digits after decimal point
temperature_3d, pressure_3d = np.around([temperature_3d, pressure_3d], decimals=2)
lat , lon = np.around([lat, lon], decimals=2)

In [ ]:
ds = xr.Dataset(data_vars={"temperature":(["x","y","day"],temperature_3d), 
                           "pressure":(["x","y","day"],pressure_3d)}, 
                coords={"lat": (["x","y"], lat), 
                        "lon": (["x","y"], lon), 
                        "day": ["day1", "day2"]})

ds

<xarray.Dataset>
Dimensions:      (x: 1, y: 4, day: 2)
Coordinates:
    lat          (x, y) float64 5.68 5.73 24.19 62.9
    lon          (x, y) float64 80.8 39.97 80.08 -63.73
  * day          (day) <U4 'day1' 'day2'
Dimensions without coordinates: x, y
Data variables:
    temperature  (x, y, day) float64 4.14 24.97 17.83 ... 31.51 -9.27 10.71
    pressure     (x, y, day) float64 1.026e+03 1.004e+03 ... 1.009e+03 1.009e+03

In [ ]:
temperature_1d, pressure_1d, lat, lon = [arr.flatten() for arr in [temperature_3d, pressure_3d, lat, lon]]    # make data 1-dimensional
day = ["day1","day2"]
idx = pd.MultiIndex(levels=[day,lat,lon], codes=[[0]*4 + [1]*4, list(range(4))*2, list(range(4))*2], names=["day","lat","lon"])

# create series
s_temperature = pd.Series(temperature_1d, index=idx)
s_pressure = pd.Series(pressure_1d, index=idx)

# create DataArrays
da_temperature = xr.DataArray.from_series(s_temperature)
da_pressure = xr.DataArray.from_series(s_pressure)

In [ ]:
ds = xr.Dataset(data_vars={"temperature": da_temperature, "pressure": da_pressure})
ds

<xarray.Dataset>
Dimensions:      (day: 2, lat: 4, lon: 4)
Coordinates:
  * day          (day) object 'day1' 'day2'
  * lat          (lat) float64 5.68 5.73 24.19 62.9
  * lon          (lon) float64 80.8 39.97 80.08 -63.73
Data variables:
    temperature  (day, lat, lon) float64 4.14 nan nan nan ... nan nan nan 10.71
    pressure     (day, lat, lon) float64 1.026e+03 nan nan ... nan nan 1.009e+03

# Data structures for multi-dimensional data

## DataArray

In [ ]:
import numpy as np
import xarray as xr

In [ ]:
rng = np.random.default_rng(seed=0)

In [ ]:
da = xr.DataArray(
    np.ones((3, 4, 2)),
    dims=("x", "y", "z"),
    name="a",
    coords={"z": [-1, 1], "u": ("x", [0.1, 1.2, 2.3])},
    attrs={"attr": "value"},
)

In this case, we used a 3x4 numpy array with all values being equal to 1, but it can be anything that either behaves like a numpy array or can be coerced to a numpy array using numpy.array.

We also passed a sequence (a tuple here, but could also be a list) containing the dimension names x, y, and z to dims. In case we have only a single dimension we can also pass just the dimension name. For example:

In [ ]:
xr.DataArray([1, 1], dims="x")

<xarray.DataArray (x: 2)>
array([1, 1])
Dimensions without coordinates: x

### coordinates

In [ ]:
da = xr.DataArray(
    np.ones((3, 4)),
    dims=("x", "y"),
    coords={
        "x": ["a", "b", "c"],
        "y": np.arange(4),
        "u": ("x", np.arange(3), {"attr1": 0}),
    },
)
da

<xarray.DataArray (x: 3, y: 4)>
array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])
Coordinates:
  * x        (x) <U1 'a' 'b' 'c'
  * y        (y) int64 0 1 2 3
    u        (x) int64 0 1 2

In [ ]:
(rng.random((180, 360)) * 400).shape

(180, 360)

In [ ]:
height = rng.random((180, 360)) * 400
xr.DataArray(
    
)

## Roundtripping and I/O （重要，可以转换成csv）
DataArray and Dataset objects are frequently created by converting from other libraries such as pandas or by reading from data storage formats such as NetCDF or zarr.

To convert from / to pandas, we can use the to_xarray methods on pandas objects or the to_pandas methods on xarray objects:

In [ ]:
series = pd.Series(np.ones((10,)), index=list("abcdefghij"))
series

a    1.0
b    1.0
c    1.0
d    1.0
e    1.0
f    1.0
g    1.0
h    1.0
i    1.0
j    1.0
dtype: float64

In [ ]:
arr = series.to_xarray()
arr

<xarray.DataArray (index: 10)>
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
Coordinates:
  * index    (index) object 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j'

## 从xarray转换到pandas

In [ ]:
arr.to_pandas()

index
a    1.0
b    1.0
c    1.0
d    1.0
e    1.0
f    1.0
g    1.0
h    1.0
i    1.0
j    1.0
dtype: float64

In [ ]:
ds = xr.Dataset(data_vars={"a": ("x", np.arange(5)), "b": (("x", "y"), np.ones((5, 4)))})

In [ ]:
ds

<xarray.Dataset>
Dimensions:  (x: 5, y: 4)
Dimensions without coordinates: x, y
Data variables:
    a        (x) int64 0 1 2 3 4
    b        (x, y) float64 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0 1.0

In [ ]:
ds.a.to_series()

x
0    0
1    1
2    2
3    3
4    4
Name: a, dtype: int64

In [ ]:
ds.b.to_series()

x  y
0  0    1.0
   1    1.0
   2    1.0
   3    1.0
1  0    1.0
   1    1.0
   2    1.0
   3    1.0
2  0    1.0
   1    1.0
   2    1.0
   3    1.0
3  0    1.0
   1    1.0
   2    1.0
   3    1.0
4  0    1.0
   1    1.0
   2    1.0
   3    1.0
Name: b, dtype: float64

In [ ]:
ds.a.to_dataframe()

,a
x,
0,0
1,1
2,2
3,3
4,4


In [ ]:
ds.to_dataframe()

a    b
x y        
0 0  0  1.0
  1  0  1.0
  2  0  1.0
  3  0  1.0
1 0  1  1.0
  1  1  1.0
  2  1  1.0
  3  1  1.0
2 0  2  1.0
  1  2  1.0
  2  2  1.0
  3  2  1.0
3 0  3  1.0
  1  3  1.0
  2  3  1.0
  3  3  1.0
4 0  4  1.0
  1  4  1.0
  2  4  1.0
  3  4  1.0

# I/O

One of Xarray’s most widely used features is its ability to read from and write to a variety of data formats. For example, Xarray can read the following formats:

- NetCDF / GRIB (via open_dataset / open_mfdataset, to_netcdf / save_mfdataset)

- Zarr (via open_zarr, to_zarr)

- GeoTIFF / GDAL rasters (via open_rasterio)


### NetCDF
The recommended way to store xarray data structures is NetCDF, which is a binary file format for self-described datasets that originated in the geosciences. Xarray is based on the netCDF data model, so netCDF files on disk directly correspond to Dataset objects.

Xarray reads and writes to NetCDF files using the open_dataset / open_dataarray functions and the to_netcdf method.

Let’s first create some datasets and write them to disk using to_netcdf, which takes the path we want to write to:

In [ ]:
ds1 = xr.Dataset(
    data_vars={
        "a": (("x", "y"), np.random.randn(4, 2)),
        "b": (("z", "x"), np.random.randn(6, 4)),
    },
    coords={
        "x": np.arange(4),
        "y": np.arange(-2, 0),
        "z": np.arange(-3, 3),
    },
)
ds2 = xr.Dataset(
    data_vars={
        "a": (("x", "y"), np.random.randn(7, 3)),
        "b": (("z", "x"), np.random.randn(2, 7)),
    },
    coords={
        "x": np.arange(6, 13),
        "y": np.arange(3),
        "z": np.arange(3, 5),
    },
)

# write datasets
ds1.to_netcdf("ds1.nc")
ds2.to_netcdf("ds2.nc")

# write dataarray
ds1.a.to_netcdf("da1.nc")

In [ ]:
xr.open_dataset('ds1.nc')

<xarray.Dataset>
Dimensions:  (x: 4, y: 2, z: 6)
Coordinates:
  * x        (x) int64 0 1 2 3
  * y        (y) int64 -2 -1
  * z        (z) int64 -3 -2 -1 0 1 2
Data variables:
    a        (x, y) float64 ...
    b        (z, x) float64 ...

In [ ]:
xr.open_dataarray("da1.nc")

<xarray.DataArray 'a' (x: 4, y: 2)>
array([[-1.253881, -0.637752],
       [ 0.907105, -1.428681],
       [-0.140069, -0.861755],
       [-0.255619, -2.798589]])
Coordinates:
  * x        (x) int64 0 1 2 3
  * y        (y) int64 -2 -1

# Working with labeled data

Learning goals:

- Use different forms of indexing to select data based on position and coordinates

- Select datatime ranges

- Interpolate data to new coordinates

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

np.random.seed(0)

In [ ]:
# axis0: x, axis1: y
np_array = np.random.randn(3,4)

np_array[1,3]

0.37816251960217356

In [ ]:
#with label based indexing:

arr = xr.DataArray(np_array,dims=('x','y'))
arr.isel(x=1,y=3)

<xarray.DataArray ()>
array(0.37816252)

In [ ]:
ds = xr.Dataset(
    {
        'a':(('x','y'),np.random.randn(3,4)),
        'b':(('x','y'),np.random.randn(3,4)),
    }
)

ds.isel(x=slice(None,2),y=slice(1,None))

<xarray.Dataset>
Dimensions:  (x: 2, y: 3)
Dimensions without coordinates: x, y
Data variables:
    a        (x, y) float64 1.202 -0.3873 -0.3023 -1.42 -1.706 1.951
    b        (x, y) float64 -0.2127 -0.8955 0.3869 -1.181 -0.02818 0.4283

In [ ]:
ds

<xarray.Dataset>
Dimensions:  (x: 3, y: 4)
Dimensions without coordinates: x, y
Data variables:
    a        (x, y) float64 1.23 1.202 -0.3873 -0.3023 ... -0.4381 -1.253 0.7775
    b        (x, y) float64 -1.614 -0.2127 -0.8955 ... 0.3025 -0.6343 -0.3627

## 创造数据集和选择

In [ ]:
data = 283 + 5*np.random.randn(5,3,4)
data

#三个维度，对应time,lan,lot

array([[[279.63769776, 281.20223419, 278.93426859, 274.36858699],
        [283.88713071, 280.99109532, 274.84900827, 285.31391128],
        [278.46350818, 283.25972698, 286.64545281, 283.64491455]],

       [[288.69700342, 276.8258709 , 285.01170821, 279.57594955],
        [278.64601425, 280.10575168, 281.44223734, 283.28082671],
        [277.1742508 , 287.50413243, 285.3283122 , 275.31878157]],

       [[290.44126097, 292.47944588, 288.89389786, 282.10037582],
        [277.64623689, 288.27225863, 280.98411527, 289.11222535],
        [284.04137489, 287.88319518, 284.78183199, 286.53286584]],

       [[283.0525001 , 291.92935247, 283.63456046, 285.00994682],
        [292.41575349, 276.26120469, 276.64757501, 287.84698354],
        [277.13438297, 292.71810593, 280.9319051 , 279.26272594]],

       [[292.61471013, 290.40257396, 292.3377948 , 287.53022329],
        [278.69387157, 292.55032477, 281.65998315, 287.01228198],
        [287.73625984, 282.22494953, 286.07039685, 287.61103336]]])

In [ ]:
temp = xr.DataArray(data)
temp

<xarray.DataArray (dim_0: 5, dim_1: 3, dim_2: 4)>
array([[[279.63769776, 281.20223419, 278.93426859, 274.36858699],
        [283.88713071, 280.99109532, 274.84900827, 285.31391128],
        [278.46350818, 283.25972698, 286.64545281, 283.64491455]],

       [[288.69700342, 276.8258709 , 285.01170821, 279.57594955],
        [278.64601425, 280.10575168, 281.44223734, 283.28082671],
        [277.1742508 , 287.50413243, 285.3283122 , 275.31878157]],

       [[290.44126097, 292.47944588, 288.89389786, 282.10037582],
        [277.64623689, 288.27225863, 280.98411527, 289.11222535],
        [284.04137489, 287.88319518, 284.78183199, 286.53286584]],

       [[283.0525001 , 291.92935247, 283.63456046, 285.00994682],
        [292.41575349, 276.26120469, 276.64757501, 287.84698354],
        [277.13438297, 292.71810593, 280.9319051 , 279.26272594]],

       [[292.61471013, 290.40257396, 292.3377948 , 287.53022329],
        [278.69387157, 292.55032477, 281.65998315, 287.01228198],
        [287.73625984, 282.22494953, 286.07039685, 287.61103336]]])
Dimensions without coordinates: dim_0, dim_1, dim_2

In [ ]:
temp = xr.DataArray(data,dims = ['time','lat','lon']) #给dims命名
temp

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[279.63769776, 281.20223419, 278.93426859, 274.36858699],
        [283.88713071, 280.99109532, 274.84900827, 285.31391128],
        [278.46350818, 283.25972698, 286.64545281, 283.64491455]],

       [[288.69700342, 276.8258709 , 285.01170821, 279.57594955],
        [278.64601425, 280.10575168, 281.44223734, 283.28082671],
        [277.1742508 , 287.50413243, 285.3283122 , 275.31878157]],

       [[290.44126097, 292.47944588, 288.89389786, 282.10037582],
        [277.64623689, 288.27225863, 280.98411527, 289.11222535],
        [284.04137489, 287.88319518, 284.78183199, 286.53286584]],

       [[283.0525001 , 291.92935247, 283.63456046, 285.00994682],
        [292.41575349, 276.26120469, 276.64757501, 287.84698354],
        [277.13438297, 292.71810593, 280.9319051 , 279.26272594]],

       [[292.61471013, 290.40257396, 292.3377948 , 287.53022329],
        [278.69387157, 292.55032477, 281.65998315, 287.01228198],
        [287.73625984, 282.22494953, 286.07039685, 287.61103336]]])
Dimensions without coordinates: time, lat, lon

In [ ]:
from pandas._libs.tslibs import period
import pandas as pd

times = pd.date_range('2018-01-01',periods=5)
times

DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
               '2018-01-05'],
              dtype='datetime64[ns]', freq='D')

In [ ]:
# sample Lon/Lan

lons = np.linspace(-120,-60,4)
lats = np.linspace(25,53,3)

In [ ]:
temp = xr.DataArray(data,coords=[times,lats,lons],dims=['time','lat','lon'])
temp

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[279.63769776, 281.20223419, 278.93426859, 274.36858699],
        [283.88713071, 280.99109532, 274.84900827, 285.31391128],
        [278.46350818, 283.25972698, 286.64545281, 283.64491455]],

       [[288.69700342, 276.8258709 , 285.01170821, 279.57594955],
        [278.64601425, 280.10575168, 281.44223734, 283.28082671],
        [277.1742508 , 287.50413243, 285.3283122 , 275.31878157]],

       [[290.44126097, 292.47944588, 288.89389786, 282.10037582],
        [277.64623689, 288.27225863, 280.98411527, 289.11222535],
        [284.04137489, 287.88319518, 284.78183199, 286.53286584]],

       [[283.0525001 , 291.92935247, 283.63456046, 285.00994682],
        [292.41575349, 276.26120469, 276.64757501, 287.84698354],
        [277.13438297, 292.71810593, 280.9319051 , 279.26272594]],

       [[292.61471013, 290.40257396, 292.3377948 , 287.53022329],
        [278.69387157, 292.55032477, 281.65998315, 287.01228198],
        [287.73625984, 282.22494953, 286.07039685, 287.61103336]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 39.0 53.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0

In [ ]:
temp.attrs['units'] = 'kelvin'
temp.attrs['standard_name'] = 'air_temperature'

temp

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[279.63769776, 281.20223419, 278.93426859, 274.36858699],
        [283.88713071, 280.99109532, 274.84900827, 285.31391128],
        [278.46350818, 283.25972698, 286.64545281, 283.64491455]],

       [[288.69700342, 276.8258709 , 285.01170821, 279.57594955],
        [278.64601425, 280.10575168, 281.44223734, 283.28082671],
        [277.1742508 , 287.50413243, 285.3283122 , 275.31878157]],

       [[290.44126097, 292.47944588, 288.89389786, 282.10037582],
        [277.64623689, 288.27225863, 280.98411527, 289.11222535],
        [284.04137489, 287.88319518, 284.78183199, 286.53286584]],

       [[283.0525001 , 291.92935247, 283.63456046, 285.00994682],
        [292.41575349, 276.26120469, 276.64757501, 287.84698354],
        [277.13438297, 292.71810593, 280.9319051 , 279.26272594]],

       [[292.61471013, 290.40257396, 292.3377948 , 287.53022329],
        [278.69387157, 292.55032477, 281.65998315, 287.01228198],
        [287.73625984, 282.22494953, 286.07039685, 287.61103336]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 39.0 53.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

### Selection

In [ ]:
temp.sel(time='2018-01-02')

<xarray.DataArray (lat: 3, lon: 4)>
array([[288.69700342, 276.8258709 , 285.01170821, 279.57594955],
       [278.64601425, 280.10575168, 281.44223734, 283.28082671],
       [277.1742508 , 287.50413243, 285.3283122 , 275.31878157]])
Coordinates:
    time     datetime64[ns] 2018-01-02
  * lat      (lat) float64 25.0 39.0 53.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

In [ ]:
#.sel has the flexibility to also perform nearest neighbor sampling, taking an optional tolerance:

from datetime import timedelta



In [ ]:
temp.sel(time='2018-01-07',method='nearest',tolerance=timedelta(days=2))

<xarray.DataArray (lat: 3, lon: 4)>
array([[292.61471013, 290.40257396, 292.3377948 , 287.53022329],
       [278.69387157, 292.55032477, 281.65998315, 287.01228198],
       [287.73625984, 282.22494953, 286.07039685, 287.61103336]])
Coordinates:
    time     datetime64[ns] 2018-01-05
  * lat      (lat) float64 25.0 39.0 53.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

In [ ]:
#Slicing with Selection

temp.sel(time=slice('2018-01-01', '2018-01-03'), lon=slice(-110, -70), lat=slice(25, 45))

<xarray.DataArray (time: 3, lat: 2, lon: 2)>
array([[[281.20223419, 278.93426859],
        [280.99109532, 274.84900827]],

       [[276.8258709 , 285.01170821],
        [280.10575168, 281.44223734]],

       [[292.47944588, 288.89389786],
        [288.27225863, 280.98411527]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 2018-01-03
  * lat      (lat) float64 25.0 39.0
  * lon      (lon) float64 -100.0 -80.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature